In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from scipy.stats import skew
from sklearn.decomposition import PCA
import statsmodels.api as sm
from sklearn.preprocessing import PolynomialFeatures
from matplotlib import pyplot as plt
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import PowerTransformer

In [2]:
train = pd.read_csv('../train.csv')
test = pd.read_csv('../test.csv')
testOriginal = pd.read_csv('../test.csv')

In [3]:
train.shape

(181507, 272)

In [4]:
test.drop('row ID', axis=1, inplace=True)

<h3>SubArea Removal<h3>

In [5]:
test.drop('sub_area', axis=1, inplace=True)
train.drop('sub_area', axis=1, inplace=True)

<h1>Categorical To Numerical<h1>

<h3>Label<h3>

In [6]:
categorical_columns = train.select_dtypes(include=['object']).columns.tolist()
print("Train: Categorical columns:", categorical_columns)

label_encoder = LabelEncoder()

for column in categorical_columns:
    train[column] = label_encoder.fit_transform(train[column])

categorical_columns_test = test.select_dtypes(include=['object']).columns.tolist()
print("Test: Categorical columns:", categorical_columns_test)

label_encoder = LabelEncoder()

for column in categorical_columns_test:
    test[column] = label_encoder.fit_transform(test[column])

Train: Categorical columns: ['product_type', 'culture_objects_top_25', 'thermal_power_plant_raion', 'incineration_raion', 'oil_chemistry_raion', 'radiation_raion', 'railroad_terminal_raion', 'big_market_raion', 'nuclear_reactor_raion', 'detention_facility_raion', 'water_1line', 'big_road1_1line', 'railroad_1line', 'ecology']
Test: Categorical columns: ['product_type', 'culture_objects_top_25', 'thermal_power_plant_raion', 'incineration_raion', 'oil_chemistry_raion', 'radiation_raion', 'railroad_terminal_raion', 'big_market_raion', 'nuclear_reactor_raion', 'detention_facility_raion', 'water_1line', 'big_road1_1line', 'railroad_1line', 'ecology']


<h1>Working<h1>

In [7]:
X = train.loc[:, train.columns != 'price_doc']
y = train[['price_doc']]

# scaler = RobustScaler()
# X = scaler.fit_transform(X)
# test = scaler.transform(test)  

In [8]:
X.shape

(181507, 270)

In [9]:
test.shape

(77789, 270)

<h1>Forward Feature Selection<h1>

In [10]:
# X2 = X[['full_sq', 'floor', 'build_count_monolith', 'industrial_km', 'trc_sqm_500',
#  'mosque_count_500', 'leisure_count_500', 'office_sqm_1000',
#  'cafe_count_1000_price_high', 'leisure_count_1000', 'power_transmission_line_km', 
#  'big_market_km', 'public_healthcare_km', 'workplaces_km']]
# test2 = test[['full_sq', 'floor', 'build_count_monolith', 'industrial_km', 'trc_sqm_500',
#  'mosque_count_500', 'leisure_count_500', 'office_sqm_1000',
#  'cafe_count_1000_price_high', 'leisure_count_1000', 'power_transmission_line_km', 
#  'big_market_km', 'public_healthcare_km', 'workplaces_km']]

<h1>Min_Max Scaling<h1>

In [11]:
scaler = MinMaxScaler()
X3 = scaler.fit_transform(X)
test3 = scaler.transform(test)  

pca = PCA(n_components=10)
principalComponents = pca.fit_transform(X3)
X3 = pd.DataFrame(data = principalComponents)

pca2 = PCA(n_components=10)
principalComponents = pca2.fit_transform(test3)
test3 = pd.DataFrame(data = principalComponents)


poly = PolynomialFeatures(2, interaction_only=True)
X3 = poly.fit_transform(X3)
test3 = poly.fit_transform(test3)


<h1>Applying Model<h1>

In [12]:
X3.shape

(181507, 56)

In [13]:
test3.shape

(77789, 56)

In [14]:



# Create and train the Lasso regression model
alpha_value = 0.5  # You can adjust the alpha parameter based on your needs
lasso_reg = Lasso(alpha=alpha_value, random_state=42)
lasso_reg.fit(X3, y)

# Print the coefficients and intercept
print("Coefficients:", lasso_reg.coef_)
print("Intercept:", lasso_reg.intercept_)

# Make predictions on the test data
y_pred_lasso = lasso_reg.predict(test3)

Coefficients: [       0.          7232531.53190027 -1530151.43052125   228545.90315635
  -611473.68396899  2853294.84783682  5216238.66244975  1364683.86607958
  2374458.58770489  2177981.89882058  -458533.49959527  1231546.15045854
  -375230.16149128    78956.52742797  -650280.1003391   -968588.06770979
  -136092.49402228  -338194.44191375  -290905.6551266   -136788.22179907
  1377305.34227679   696171.38760911  3994528.99039762 -1642977.77884986
 -1514751.48089968  1028337.4635112  -1356147.97850626   960246.18003633
   270578.79581666  1051887.13179756  1004970.76032357 -1299786.45904447
   345667.47138643   579336.73232011   524089.24709184  -366111.75371978
   774859.75836871  -769054.57404511    70170.51682722  1355221.57142822
  1835353.53061448 -2891024.51769578 -5701561.41547282  2936978.72284978
  -685866.93953479  3458890.69265687   809378.90275066 -1048839.27231327
  2800972.79955804 -2319628.98983441  -270675.104013     824193.45135259
  3150735.55997584  -138957.08213791 

In [15]:
# Save the predictions to a CSV file
result_df_lasso = pd.DataFrame({'row ID': testOriginal['row ID'], 'price_doc': y_pred_lasso.flatten()})
result_df_lasso.to_csv('Day_6_Abdullah_Maqsood_24448_Submission_9.csv', index=False)